<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb

In [12]:
train = pd.read_csv('../../data/flight_delays_train.csv')
test = pd.read_csv('../../data/flight_delays_test.csv')

In [13]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [14]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [15]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [16]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.67956914653526068

In [17]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [20]:
#X_train_part, X_valid, y_train_part, y_valid
dtrain = xgb.DMatrix(X_train_part, y_train_part)
dtest = xgb.DMatrix(X_valid, y_valid)

In [24]:
params = {
    'objective':'binary:logistic',
    'max_depth': 3,
    'silent': 1,
    'eta': 1
}

num_rounds = 500

In [25]:
xgb_model = xgb.train(params, dtrain, num_rounds)

In [26]:
watchlist  = [(dtest,'test'), (dtrain,'train')] # native interface only
xgb_model = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-error:0.188433	train-error:0.190657
[1]	test-error:0.188433	train-error:0.190657
[2]	test-error:0.188167	train-error:0.189829
[3]	test-error:0.1867	train-error:0.186857
[4]	test-error:0.186467	train-error:0.186471
[5]	test-error:0.186433	train-error:0.186014
[6]	test-error:0.1862	train-error:0.185629
[7]	test-error:0.186433	train-error:0.185786
[8]	test-error:0.186233	train-error:0.185729
[9]	test-error:0.186233	train-error:0.185486
[10]	test-error:0.1864	train-error:0.185486
[11]	test-error:0.1862	train-error:0.185414
[12]	test-error:0.1862	train-error:0.185529
[13]	test-error:0.186233	train-error:0.185486
[14]	test-error:0.186	train-error:0.185457
[15]	test-error:0.186067	train-error:0.185386
[16]	test-error:0.185967	train-error:0.185414
[17]	test-error:0.185867	train-error:0.185286
[18]	test-error:0.185833	train-error:0.185257
[19]	test-error:0.185567	train-error:0.1852
[20]	test-error:0.185367	train-error:0.184943
[21]	test-error:0.1853	train-error:0.184714
[22]	test-error

[181]	test-error:0.187067	train-error:0.178571
[182]	test-error:0.187233	train-error:0.1786
[183]	test-error:0.187267	train-error:0.178486
[184]	test-error:0.187267	train-error:0.178471
[185]	test-error:0.187233	train-error:0.178357
[186]	test-error:0.187433	train-error:0.178457
[187]	test-error:0.1876	train-error:0.178443
[188]	test-error:0.187433	train-error:0.178529
[189]	test-error:0.187367	train-error:0.1784
[190]	test-error:0.1874	train-error:0.178471
[191]	test-error:0.187367	train-error:0.178443
[192]	test-error:0.187333	train-error:0.178486
[193]	test-error:0.187533	train-error:0.1784
[194]	test-error:0.187533	train-error:0.178343
[195]	test-error:0.187533	train-error:0.178329
[196]	test-error:0.187533	train-error:0.178371
[197]	test-error:0.187467	train-error:0.178357
[198]	test-error:0.187467	train-error:0.178386
[199]	test-error:0.187433	train-error:0.1783
[200]	test-error:0.187467	train-error:0.1783
[201]	test-error:0.1874	train-error:0.178343
[202]	test-error:0.1875	train

[360]	test-error:0.189333	train-error:0.173557
[361]	test-error:0.189267	train-error:0.173471
[362]	test-error:0.189433	train-error:0.173486
[363]	test-error:0.189267	train-error:0.173657
[364]	test-error:0.1894	train-error:0.1737
[365]	test-error:0.189467	train-error:0.173586
[366]	test-error:0.1894	train-error:0.1737
[367]	test-error:0.189333	train-error:0.173657
[368]	test-error:0.189533	train-error:0.173657
[369]	test-error:0.189767	train-error:0.173386
[370]	test-error:0.1898	train-error:0.1734
[371]	test-error:0.189733	train-error:0.173443
[372]	test-error:0.1898	train-error:0.173471
[373]	test-error:0.19	train-error:0.1734
[374]	test-error:0.189967	train-error:0.173543
[375]	test-error:0.1899	train-error:0.173486
[376]	test-error:0.1899	train-error:0.173486
[377]	test-error:0.1899	train-error:0.1735
[378]	test-error:0.189867	train-error:0.1734
[379]	test-error:0.189733	train-error:0.173429
[380]	test-error:0.189667	train-error:0.173414
[381]	test-error:0.189767	train-error:0.173

In [27]:
preds_prob = xgb_model.predict(dtest)

In [29]:
from sklearn.metrics import accuracy_score, f1_score

In [31]:
predicted_labels = preds_prob > 0.5
print("Accuracy and F1 on the test set are: {} and {}".format(
    round(accuracy_score(y_valid, predicted_labels), 3),
    round(f1_score(y_valid, predicted_labels), 3)))

Accuracy and F1 on the test set are: 0.81 and 0.168
